Migrare i dati da csv su Storage Account a tabella Sql Server su Azure

In [0]:
from pyspark.sql.types import *

In [0]:
%fs ls /mnt/sa/Blob

In [0]:

storageAccount  = "sadatabricksadlsgen2gr"
storageKey = "odN94b75Cdh/mKKPjDYCo9AQveP5cjMNY/xMctEsRTJGCSS9dfqjQw6PCqvDqdmDHEsn7zl+Z+jr+AStYZShBw=="
mountPoint = "/mnt/sa/Blob"
storageEndPoint = "wasbs://rawdata@{}.blob.core.windows.net".format(storageAccount)
storageConn = "fs.azure.account.key.{}.blob.core.windows.net".format(storageAccount)

In [0]:

try:
    dbutils.fs.mount(
        source        = storageEndPoint,
        mount_point   = mountPoint,
        extra_configs = { storageConn : storageKey }
    )
except:
    print("Il sa è già stato montato.... " + mountPoint)

In [0]:
# Creo un tipo Struct per indicare i metadati utili per tipizzare le colonne dei dati caricati
customSchema = StructType([
    StructField("C_CUSTKEY"    , IntegerType()),
    StructField("C_NAME"       , StringType()),
    StructField("C_ADDRESS"    , StringType()),
    StructField("C_NATIONKEY"  , ShortType()),
    StructField("C_PHONE"      , StringType()),
    StructField("C_ACCTBAL"    , DoubleType()),
    StructField("C_MKTSEGMENT" , StringType()),
    StructField("C_COMMENT"    , StringType())
])

saFolderCsv = "{}/Customer/csvFiles".format(mountPoint)
display(saFolderCsv)
customerDF = spark.read.format("csv").option("header", True).schema(customSchema).load(saFolderCsv)
display(customerDF.limit(10))

In [0]:
# connection string
logicalServername = "sqlservecorsodbgr.database.windows.net"
databaseName      = "CorsoDB01"
tableName         = "dbo.CUSTOMERS"
userName          = "admingr"
password          = "1234,qazwsx" 
jdbcUrl = "jdbc:sqlserver://{0}:{1};database={2}".format(logicalServername, 1433, databaseName)
connectionProperties = {
  "user"     : userName,
  "password" : password,
  "driver"   : "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

sqlServerReadDF = spark.read.jdbc(
    jdbcUrl,
    table      = tableName,
    properties = connectionProperties
)

display(sqlServerReadDF.printSchema())

display(sqlServerReadDF.limit(10))
